In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

from tqdm import tqdm


In [ ]:
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install squad_torch_bert

In [ ]:
from deeppavlov import configs, build_model

squad_model = build_model(configs.squad.squad_torch_bert, download=True)

In [37]:
with open('CONLL2003/eng_train.txt', 'r', encoding='utf8') as f:
  text = f.read()
  f.close()

In [38]:
import re

In [39]:
text = re.sub('I-MISC', 'misc', text)
text = re.sub('I-LOC', 'location', text) 
text = re.sub('I-PER', 'person', text)
text = re.sub('I-ORG', 'organisation', text)

In [40]:
texts = [text.split('\n') for text in [text for text in text.split('\n\n')]]

In [42]:
len(texts) 

14987

In [ ]:
true_org = []
true_person = []
pred_org = []
pred_person = []

true_loc = []
true_misc = []
pred_loc = []
pred_misc = []
n = 0
for text in texts:
  n+=1
  print(n)
  per = []
  org = []
  loc = []
  misc = []
  clean_text = ''
  for e in text:
    token_full = e.split(' ')
    ind = len(token_full) - 1
    ent_name = token_full[ind]
    token = token_full[0]
    if token in "'s'.,":
      clean_text += token
    if token not in "'s'.,":
      clean_text += " " +  token
    if ent_name == 'person':
      per.append(token)
    if ent_name == 'organisation':
      org.append(token) 
    if ent_name == 'location':
      loc.append(token)
    if ent_name == 'misc':
      misc.append(token)

  result_person = squad_model([clean_text], 
      ['person'])
  result_org = squad_model([clean_text], 
      ['organisation'])
  result_loc = squad_model([clean_text], 
      ['location'])
  result_misc = squad_model([clean_text], 
      ['entity'])
  
  result_person = ' '.join(result_person[0])
  result_org = ' '.join(result_org[0])
  result_loc = ' '.join(result_loc[0])
  result_misc = ' '.join(result_misc[0])
  
  orig_pers = ' '.join(per)
  orig_orgs = ' '.join(org)
  orig_loc = ' '.join(loc)
  orig_misc = ' '.join(misc)

  true_person.append(orig_pers)
  true_org.append(orig_orgs)
  true_loc.append(orig_loc)
  true_misc.append(orig_misc)

  pred_person.append(result_person)
  pred_org.append(result_org)
  pred_loc.append(result_loc)
  pred_misc.append(result_misc)



Только непустая разметка

In [50]:
def precision_sc(pred, true, entity_name):
  precision = 0
  n = 0
  for pred_, true_ in zip(pred, true):
    if true_:
      n += 1
      pred_ = pred_.split(' ')
      true_ = true_.split(' ')
      tp = set(pred_).intersection(set(true_))
      fp = len(pred_) - len(tp)
      precision += len(tp)/(len(tp) + fp)
    else:
      continue

  print(entity_name, 'precision:', precision/n)

In [51]:
def recall_sc(pred, true, entity_name):
  recall = 0
  n = 0
  for true_, pred_ in zip(true, pred):
    if true_:
      n += 1
      pred_ = pred_.split(' ')
      true_ = true_.split(' ')
      tp = set(pred_).intersection(set(true_))
      fn = len(true_) - len(tp)
      recall += len(tp)/(len(tp) + fn)
    else:
      continue

  print(entity_name, 'recall:', recall/n)

In [52]:
precision_sc(pred_person, true_person, "person")
precision_sc(pred_org, true_org, "organisation")
precision_sc(pred_loc, true_loc, "location")
precision_sc(pred_misc, true_misc, "misc")

recall_sc(pred_person, true_person,'person')
recall_sc(pred_org, true_org, "organisation")
recall_sc(pred_loc, true_loc, "location")
recall_sc(pred_misc, true_misc, "misc")

person precision: 0.5675340189611583
organisation precision: 0.5148745483580227
location precision: 0.3838386614283244
misc precision: 0.1920936596038542
person recall: 0.6642088935654185
organisation recall: 0.7383926485790446
location recall: 0.6203857798357507
misc recall: 0.5727783366867872


Все примеры

In [55]:
def precision_2(pred, true, entity_name):
  precision = 0
  for pred_, true_ in zip(pred, true):
    pred_ = pred_.split(' ')
    true_ = true_.split(' ')
    tp = set(pred_).intersection(set(true_))
    fp = len(pred_) - len(tp)
    precision += len(tp)/(len(tp) + fp)

  print(entity_name, 'precision:', precision/len(true))

def recall_2(pred, true, entity_name):
  recall = 0
  for true_, pred_ in zip(true, pred):
    pred_ = pred_.split(' ')
    true_ = true_.split(' ')
    tp = set(pred_).intersection(set(true_))
    fn = len(true_) - len(tp)
    recall += len(tp)/(len(tp) + fn)

  print(entity_name, 'recall:', recall/len(true))

In [56]:
precision_2(pred_person, true_person, "person")
precision_2(pred_org, true_org, "organisation")
precision_2(pred_loc, true_loc, "location")
precision_2(pred_misc, true_misc, "misc")

recall_2(pred_person, true_person,'person')
recall_2(pred_org, true_org, "organisation")
recall_2(pred_loc, true_loc, "location")
recall_2(pred_misc, true_misc, "misc")

person precision: 0.2714903759869978
organisation precision: 0.21249946976167688
location precision: 0.18755860526743273
misc precision: 0.045724207220337675
person recall: 0.2996987717062507
organisation recall: 0.28091059445066274
location recall: 0.26848054268485294
misc recall: 0.11425608543277187
